<a href="https://colab.research.google.com/github/johnathan2012/Programming-iOS-Book-Examples/blob/master/GPT4Dev_ch06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6 讓 AI 幫 AI－自動串接流程

## 6-1 從 ChatGPT 外掛得到的啟示

### 準備工作

安裝必要的套件與匯入相關模組後建立用戶端物件

In [ ]:
!pip install openai
!pip install rich
!pip install googlesearch-python
import openai
from googlesearch import search
from google.colab import userdata
from rich import print as pprint
client = openai.OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

### 搭配串流/非串流模式的工具函式

In [ ]:
def get_reply_g(messages, stream=False, json_format=False):
    try:
        json_msg = [{
            'role': 'system', 'content': '請用 JSON 回覆'
        }] if json_format else []
        response = client.chat.completions.create(
            model = "gpt-4-1106-preview",
            messages = messages + json_msg,
            stream = stream,
            response_format = {
                'type': "json_object" if json_format else 'text'
            }
        )
        if stream: # 串留模式下交給輔助函式取得完整文字
            for res in response:
                yield res.choices[0].delta.content or ''
        else:      # 非串流模式下可直接取得完整回覆文字
            yield response.choices[0].message.content
    except openai.APIError as err:
        reply = f"發生錯誤\n{err.message}"
        print(reply)
        yield reply

In [ ]:
# 測試非串流模式的情況
for reply in get_reply_g([{'role':'user', 'content':'你好'}]):
    print(reply)

In [ ]:
# 測試串流模式的情況
for msg in get_reply_g([{'role':'user', 'content':'你好'}], True):
    print(msg, end='')

In [ ]:
# 測試 JSON 格式輸出
for reply in get_reply_g(
    [{'role':'user', 'content':'你好'}],
    json_format=True
):
    print(reply)

## 6-2 由 AI 自動判斷要額外進行的工作

### 撰寫判斷是否需要搜尋的工具函式

In [ ]:
# 用來詢問是否需要搜尋才能回覆問題的樣板
# 要求 AI 以 JSON 格式回覆 Y/N 以及建議的搜尋關鍵字
template_google = '''
如果我想知道以下這件事, 請確認是否需要網路搜尋才做得到？

```
{}
```

如果需要, 請以下列 JSON 格式回答我, 除了 JSON 格式資料外,
不要加上額外資訊, 就算你知道答案, 也不要回覆：

```
{{
    "search":"Y",
    "keyword":"你建議的搜尋關鍵字"
}}
```
如果不需要, 請以下列 JSON 格式回答我：

```
{{
    "search":"N",
    "keyword":""
}}
'''

In [ ]:
# 利用目前歷史紀錄以及樣板內容詢問是否需要搜尋才能回覆問題
# 如果需要回覆, 也同時取得 AI 推薦的搜尋關鍵字
def check_google(hist, msg, verbose=False):
    reply = get_reply_g(
        hist + [{  # 加入歷史紀錄 AI 才能推薦正確的關鍵字
            "role": "user",
            "content": template_google.format(msg)
        }], json_format=True)
    for ans in reply:pass
    if verbose: print(ans)
    return ans

In [ ]:
# 測試需要搜尋的狀況
ans = check_google(
    [], '2023 金馬影后是誰？', True
)
# 測試可能不需要搜尋的狀況
ans = check_google(
    [], '新冠疫情是哪一年開始的？', True
)
# 測試沒有前文脈絡的狀況
ans = check_google(
    [], '那台灣呢？', True
)
# 測試包含前文脈絡的狀況
ans = check_google(
    [{'role':'assistant', 'content': '印度空污好嚴重'}],
    '那台灣呢？', True
)

In [ ]:
def google_res(user_msg, num_results=5, verbose=False):
    content = "以下為已發生的事實：\n"                # 強調資料可信度
    for res in search(user_msg, advanced=True,    # 一一串接搜尋結果
                      num_results=num_results,
                      lang='zh-TW'):
        content += f"標題：{res.title}\n" \
                    f"摘要：{res.description}\n\n"
    content += "請依照上述事實回答以下問題：\n"        # 下達明確指令
    if verbose:
        print('------------')
        print(content)
        print('------------')
    return content

In [ ]:
res = google_res('2023 金馬獎影后是誰？', 2, verbose=True)

### 可自行判斷是否進行網路搜尋的聊天程式

In [ ]:
import json
hist = []       # 歷史對話紀錄
backtrace = 2   # 記錄幾組對話

def chat_g(sys_msg, user_msg, stream=False, verbose=False):
    global hist
    messages = [{'role':'user', 'content':user_msg}]
    ans = json.loads(check_google(hist, user_msg,
                                  verbose=verbose))
    if ans['search'] == 'Y':
        print(f'嘗試透過網路搜尋：{ans["keyword"]}....')
        res = google_res(ans['keyword'], verbose=verbose)
        messages = [{'role':'user', 'content': res + user_msg}]

    replies = get_reply_g(            # 使用搜尋版的函式
        hist        # 先提供歷史紀錄
        + messages  # 再提供搜尋結果及目前訊息
        + [{"role": "system", "content": sys_msg}],
        stream)
    reply_full = ''
    for reply in replies:
        reply_full += reply
        yield reply

    hist.append({"role":"user", "content":user_msg})
    hist.append({"role":"assistant", "content":reply_full})
    hist = hist[-2 * backtrace:] # 保留最新對話

In [ ]:
sys_msg = input("你希望ㄟ唉扮演：")
if not sys_msg.strip(): sys_msg = '使用繁體中文的小助理'
print()

while True:
    msg = input("你說：")
    if not msg.strip(): break
    print(f"{sys_msg}：", end = "")
    # 不論是字串或是生成器, 都可以適用 for...in 迴圈
    for reply in chat_g(sys_msg, msg, stream=False):
        print(reply, end = "")
    print('\n')
hist = []

## 6-3 可建構外掛系統的 Function Calling 機制

**Function calling 機制**

Function calling 機制可以讓我們提供可用函式的規格, 由 AI 幫我們判斷是否需要叫用其中的函式。

### 告知語言模型可用的外部工具函式

In [ ]:
response = client.chat.completions.create(
    model = "gpt-4-1106-preview",
    messages = [{"role":"user", "content":"2023 金馬獎影后是誰？"}],
    tools = [{ # 可用的函式清單
        "type":"function",
        "function": {
            "name": "google_res",                     # 函式名稱
            "description": "取得 Google 搜尋結果",      # 函式說明
            "parameters": {
                "type": "object",
                "properties": {
                    "user_msg": {                     # 參數名稱
                        "type": "string",             # 資料型別
                        "description": "要搜尋的關鍵字", # 參數說明
                    }
                },
                "required": ["user_msg"],             # 必要參數
            },
        }
    }],
    tool_choice = "auto")       # 請 AI 判斷是否需要叫用函式

若 API 判斷需要叫用你描述的函式, 會在回覆中以 function_call 項目描述要叫用的函式名稱與參數值。

### 取得語言模型的建議

In [ ]:
pprint(response)

In [ ]:
tool_call = response.choices[0].message.tool_calls[0]
func_name = tool_call.function.name
import json
args = json.loads(tool_call.function.arguments)
arg_val = args.popitem()[1]
print(f'{func_name}("{arg_val}")')

### 執行函式並傳回結果

你必須自行叫用函式, 並且將執行結果透過 function 角色的訊息傳回

In [ ]:
# 用來過濾掉訊息中 function_call 欄位的函式
# 由於 function_call 在請求中和 tool_choice 是同樣的功能
# 如果一併傳回會造成錯誤, 所以我們這裡特別濾掉
# def make_tool_back_msg(tool_msg):
#     msg_json = tool_msg.model_dump()
#     tool_back_msg = {
#         'content': msg_json['content'],
#         'role': msg_json['role'],
#         'tool_calls': msg_json['tool_calls']
#     }
#     return tool_back_msg

In [ ]:
# 用來過濾掉訊息中 function_call 欄位的函式
# 由於 function_call 在請求中和 tool_choice 是同樣的功能
# 如果一併傳回會造成錯誤, 所以我們這裡特別濾掉
def make_tool_back_msg(tool_msg):
    msg_json = tool_msg.model_dump()
    del msg_json['function_call']
    return msg_json

In [ ]:
response = client.chat.completions.create(
    model='gpt-4-1106-preview',
    messages=[
        {"role":"user", "content":"2023 金馬獎影后是誰？"},
        # 傳回 AI 傳給我們的 function calling 結果
        make_tool_back_msg(response.choices[0].message),
        {   # 以 function 角色加上 name 屬性指定函式名稱傳回執行結果
            "tool_call_id": tool_call.id, # 叫用函式的識別碼
            "role": "tool", # 以工具角色送出回覆
            "name": func_name, # 叫用的函式名稱
            "content": eval(f'{func_name}("{arg_val}")') # 函式傳回值
        }
    ]
)

In [ ]:
print(response.choices[0].message.content)

### 同時叫用多個函式 (parallel function calling)

2023/11/06 之後的模型支援單次對話可以要求執行多個函式呼叫：

In [ ]:
response = client.chat.completions.create(
    model = "gpt-4-1106-preview",
    messages = [{
        "role":"user",
        "content":"2023 金曲獎歌后和金馬獎影后各是誰？"}],
    tools = [{ # 可用的函式清單
        "type":"function",
        "function": {
            "name": "google_res",                     # 函式名稱
            "description": "取得 Google 搜尋結果",      # 函式說明
            "parameters": {
                "type": "object",
                "properties": {
                    "user_msg": {                     # 參數名稱
                        "type": "string",             # 資料類型
                        "description": "要搜尋的關鍵字", # 參數說明
                    }
                },
                "required": ["user_msg"],             # 必要參數
            },
        }
    }],
    tool_choice = "auto")       # 請 AI 判斷是否需要叫用函式

In [ ]:
pprint(response)

In [ ]:
def make_func_messages(tool_calls):
    messages = []
    for tool_call in tool_calls:
        func = tool_call.function
        args_val = json.loads(func.arguments).popitem()[1]
        print(f'{func.name}("{args_val}")')
        messages.append({
            "tool_call_id": tool_call.id, # 叫用函式的識別碼
            "role": "tool", # 以工具角色送出回覆
            "name": func.name, # 叫用的函式名稱
            "content": eval(f'{func.name}("{args_val}")') # 傳回值
        })
    return messages

In [ ]:
msges = make_func_messages(response.choices[0].message.tool_calls)
pprint(msges)

In [ ]:
response = client.chat.completions.create(
    model='gpt-4-1106-preview',
    messages=[{
        "role":"user",
        "content":"2023 金曲獎歌后和金馬獎影后各是誰？"},
        # 傳回 AI 傳給我們的 function calling 結果
        make_tool_back_msg(response.choices[0].message),
    ] + msges
)

In [ ]:
print(response.choices[0].message.content)

### 以串流方式使用 function calling

In [ ]:
response = client.chat.completions.create(
    model = "gpt-4-1106-preview",
    messages = [{"role":"user",
                 "content":"2023 金馬獎影后和金曲獎歌后各是誰？"}],
    tools = [{
        "type": "function",                           # 工具類型
        "function": {
            "name": "google_res",                     # 函式名稱
            "description": "取得 Google 搜尋結果",      # 函式說明
            "parameters": {
                "type": "object",
                "properties": {
                    "user_msg": {                     # 參數名稱
                        "type": "string",
                        "description": "要搜尋的關鍵字", # 參數說明
                    }
                },
                "required": ["user_msg"],
            },
        }
    }],
    tool_choice = "auto", # 請 AI 判斷是否需要使用工具
    stream=True
)

傳回結果一樣是生成器

In [ ]:
for chunk in response:
    pprint(chunk)

## 6-4 建立 API 外掛系統

### 建立外部工具函式參考表

建立以 function calling 為基礎的外掛機制。<br>
建立結構化的函式表格。

In [ ]:
tools_table = [             # 可用工具表
    {                       # 每個元素代表一個工具
        "chain": True,      # 工具執行結果是否要再傳回給 API
        "func": google_res, # 工具對應的函式
        "spec": {           # function calling 需要的工具規格
            "type": "function",
            "function": {
                "name": "google_res",
                "description": "取得 Google 搜尋結果",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "user_msg": {
                            "type": "string",
                            "description": "要搜尋的關鍵字",
                        }
                    },
                    "required": ["user_msg"],
                },
            }
        }
    }
]

### 建立協助 function calling 的工具函式
依據回應內容自動叫用對應函式：

In [ ]:
def call_tools(tool_calls, tools_table):
    res = ''
    msg = []
    for tool_call in tool_calls:
        func = tool_call.function
        func_name = func.name
        args = json.loads(func.arguments)
        for f in tools_table:  # 找出包含此函式的項目
            if func_name == f['spec']['function']['name']:
                print(f"嘗試叫用：{func_name}(**{args})")
                val = f['func'](**args)
                if f['chain']: # 要將結果送回模型
                    msg.append({
                        'tool_call_id': tool_call.id,
                        'role': 'tool',
                        'name': 'func_name',
                        'content': val
                    })
                else:
                    res += str(val)
                break
    return msg, res

In [ ]:
def get_tool_calls(messages, stream=False, tools_table=None,
                  **kwargs):
    model = 'gpt-4-1106-preview' # 設定模型
    if 'model' in kwargs: model = kwargs['model']

    tools = {}
    if tools_table: # 加入工具表
        tools = {'tools':[tool['spec'] for tool in tools_table]}

    response = client.chat.completions.create(
        model = model,
        messages = messages,
        stream = stream,
        **tools
    )

    if not stream: # 非串流模式
        msg = response.choices[0].message
        if msg.content == None: # function calling 的回覆
            return msg.tool_calls, None # 取出叫用資訊
        return None, response # 一般回覆

    tool_calls = [] # 要呼叫的函式清單
    prev = None
    for chunk in response:
        delta = chunk.choices[0].delta
        if delta.content != None: # 一般回覆 (非 function calling)
            return None, response # 直接返回結果
        if delta.tool_calls:      # 不是頭/尾的 chunk
            curr = delta.tool_calls[0]
            if curr.function.name:       # 單一 call 開始
                prev = curr              # 取得工具名稱
                tool_calls.append(curr)  # 加入串列
            else: # 串接引數內容
                prev.function.arguments += curr.function.arguments
    return tool_calls, None

In [ ]:
tool_calls, reply = get_tool_calls(
    messages = [{'role':'user', 'content':'2023 金曲歌王是哪位？'}]
)

print(tool_calls)
print(reply and reply.choices[0].message.content)

### 建立 function_calling 版的 get_reply_f() 函式

In [ ]:
def get_reply_f(messages, stream=False, tools_table=None, **kwargs):
    try:
        tool_calls, response = get_tool_calls(
            messages, stream, tools_table, **kwargs)
        if tool_calls:
            tool_messages, res = call_tools(tool_calls, tools_table)
            tool_calls_messeges = []
            for tool_call in tool_calls:
                tool_calls_messeges.append(tool_call.model_dump())
            if tool_messages:  # 如果需要將函式執行結果送回給 AI 再回覆
                messages += [ # 必須傳回原本 function_calling 的內容
                    {
                        "role": "assistant", "content": None,
                        "tool_calls": tool_calls_messeges
                    }]
                messages += tool_messages
                # pprint(messages)
                yield from get_reply_f(messages, stream,
                                       tools_table, **kwargs)
            else:      # chain 為 False, 以函式叫用結果當成模型生成內容
                yield res
        elif stream:   # 不需叫用函式但使用串流模式
            for chunk in response:
                yield chunk.choices[0].delta.content or ''
        else:          # 不需叫用函式也沒有使用串流模式
            yield response.choices[0].message.content
    except openai.APIError as err:
        reply = f"發生錯誤\n{err.message}"
        print(reply)
        yield reply

In [ ]:
# 測試非串流方式 function_calling 功能
for chunk in get_reply_f(
    [{"role":"user", "content":"2023 金馬講影后是誰？"}],
    tools_table=tools_table):
    print(chunk)

In [ ]:
# 測試串流方式 function_calling 功能
for chunk in get_reply_f(
    [{"role":"user", "content":"2023 金馬講影后是誰？"}],
    stream=True,
    tools_table=tools_table):
    print(chunk, end='')

In [ ]:
# 測試非串流、無 function calling 功能
for chunk in get_reply_f(
    [{"role":"user", "content":"2023 金馬講影后是誰？"}]):
    print(chunk)

In [ ]:
# 測試串流、無 function calling 功能
for chunk in get_reply_f(
    [{"role":"user", "content":"2023 金馬講影后是誰？"}],
    stream=True):
    print(chunk, end='')

In [ ]:
# 測試串流方式 function_calling 功能
for chunk in get_reply_f(
    [{"role":"user", "content":"2023 金曲獎歌后和金馬獎影后各是誰？"}],
    stream=True,
    tools_table=tools_table):
    print(chunk, end='')

### 建立 function calling 版本的 chat_f() 函式

In [ ]:
hist = []       # 歷史對話紀錄
backtrace = 2   # 記錄幾組對話

def chat_f(sys_msg, user_msg, stream=False, **kwargs):
    global hist

    replies = get_reply_f(    # 使用函式功能版的函式
        hist                  # 先提供歷史紀錄
        + [{"role": "user", "content": user_msg}]
        + [{"role": "system", "content": sys_msg}],
        stream, tools_table, **kwargs)
    reply_full = ''
    for reply in replies:
        reply_full += reply
        yield reply

    hist += [{"role":"user", "content":user_msg},
             {"role":"assistant", "content":reply_full}]
    hist = hist[-2 * backtrace:] # 留下最新的對話

In [ ]:
sys_msg = input("你希望ㄟ唉扮演：")
if not sys_msg.strip(): sys_msg = '使用繁體中文的小助理'
print()
while True:
    msg = input("你說：")
    if not msg.strip(): break
    print(f"{sys_msg}：", end = "")
    for reply in chat_f(sys_msg, msg, stream=True):
        print(reply, end = "")
    print('\n')
hist = []